In [ ]:
###
###テキスト分析3_セル1
###

#シェイクスピアの作品分類
#テキストがGoogle Collaborator のファイルの保存位置に入る
import os
folder='texts'
if not os.path.isdir(folder):
  os.mkdir(folder)
!wget -O $folder/shakespeares-works_TXT_FolgerShakespeare.zip https://flgr.sh/txtfssAlltxt?_ga=2.104281097.80727893.1702953543-617012875.1702953542
!unzip -d $folder -o texts/shakespeares-works_TXT_FolgerShakespeare.zip
#終わりの何行を比較対象とするか
amount_of_last_lines=50



In [ ]:
###
###テキスト分析3_セル2
###

#テクストを読み取り、前処理
import glob
import re
#前処理済みのテクストを格納する変数
titles=[]
plays=[]
#詩作品は除外
exclude_poems=['the-phoenix-and-turtle','venus-and-adonis','shakespeares-sonnets','lucrece']
#ディレクトリの中のテクストを一つずつ読み込む　例外：shakespeares-sonnets
for v in glob.glob(folder+'/*.txt'):
  #詩は除外
  title=v[len(folder)+1:v.find('_')]
  if title in exclude_poems:
    continue
  with open(v) as f:
    text=f.readlines()
  print(v)
#前処理1: 前書き（テクストの説明から登場人物までINDUCTION含む）とあとがきEPILOGUEを消す
  key=['ACT 1\n']
  text2=[]
  flag=0
  for i in text:
    if i in key:
      flag=1
    #ACT 1 より前までとEPILOGUEより後が消える
    if re.search(r'^EPILOGUE.*\n$', i)!=None:
      flag=0
    if flag!=0:
      text2.append(i)
  text3=[]
  togaki_flag=False
#前処理2: ACT1 Scene1(=, Chorus含む)といった文字を消す
  for i in text2:
    if re.search(r'^ACT [0-9].*\n$', i)!=None:
      i=''
    if re.search(r'^Scene [0-9].*\n$', i)!=None:
      i=''
    if re.search(r'^==.*\n$', i)!=None:
      i=''
    if re.search(r'^[0-9].*Chorus\n$', i)!=None:
      i=''
#前処理3: ト書き[]を消す
    #ト書き[]が1行内にある場合
    m=re.findall('\[[^\]]+\]',i)
    for j in m:
      i=i.replace(j,'')
    #ト書き[]が複数行にわたってある場合
    m=i.find('[')
    if m!=-1:
      i=re.sub('\[.*','',i)
      togaki_flag=True
    if togaki_flag:
      m=i.find(']')
      #その行全体がまだト書きの場合
      if m==-1:
        i=''
      #その行に終わりの]がある場合
      if m!=-1:
        i=i[m+1:]
        togaki_flag=False
#前処理4: 人物名を消す THESEUS  など。※ I に注意
    #人物の名前で始まる行
    m=re.search(r'^[A-Z]* *', i)
    if m!=None and len(m.group())>2:
      i=re.sub('^[A-Z]* *','', i)
    #人物の名前だけで改行している場合
    m=re.search(r'^[A-Z]*$', i)
    if m!=None and len(m.group())>1:
      i=re.sub('^[A-Z]*$','', i)
#前処理5: その他不要文字
    i=re.sub('\t','', i)
#前処理6: 改行を消す
    if i!='\n' and i!='':
      text3.append(i)
  titles.append(title)
  plays.append(text3)

#確認
print(titles)
print(plays[0])

In [ ]:
###
###テキスト分析3_セル3
###

#分析には（終わりの50行）を使う。改行を除いてスペースでつなげる。
plays_short=[]
for i in plays:
  k=''
  for j in i[-amount_of_last_lines:]:
  #for j in i:
    k=k+j.replace('\n',' ')
  plays_short.append(k)
print(titles[0])
print(plays_short[0])
print(len(plays_short))

In [ ]:
###
###テキスト分析3_セル4
###

#機械学習用のラベルの作成。悲劇=1と喜劇=0に分ける。歴史劇は悲劇とする。
#hamletだけとっておく
labels_orig = {'alls-well-that-ends-well':	0,
	'a-midsummer-nights-dream':	0,
	'antony-and-cleopatra':	1,
	'as-you-like-it':	0,
	'coriolanus':	1,
	'cymbeline':	0,
	'hamlet':	1,
	'henry-iv-part-1':	1,
	'henry-iv-part-2':	1,
	'henry-v':	1,
	'henry-viii':	1,
	'henry-vi-part-1':	1,
	'henry-vi-part-2':	1,
	'henry-vi-part-3':	1,
	'julius-caesar':	1,
	'king-john':	1,
	'king-lear':	1,
	'loves-labors-lost':	0,
	'macbeth':	1,
	'measure-for-measure':	0,
	'much-ado-about-nothing':	0,
	'othello':	1,
	'pericles':	0,
	'richard-ii':	1,
	'richard-iii':	1,
	'romeo-and-juliet':	1,
	'the-comedy-of-errors':	0,
	'the-merchant-of-venice':	0,
	'the-merry-wives-of-windsor':	0,
	'the-taming-of-the-shrew':	0,
	'the-tempest':	0,
	'the-two-gentlemen-of-verona':	0,
	'the-two-noble-kinsmen':	0,
	'the-winters-tale':	0,
	'timon-of-athens':	1,
	'titus-andronicus':	1,
	'troilus-and-cressida':	0,
	'twelfth-night':	0}

'''
#悲劇、喜劇、歴史劇の３つに分けた場合
#labels_orig = {'alls-well-that-ends-well':	0,
	'a-midsummer-nights-dream':	0,
	'antony-and-cleopatra':	1,
	'as-you-like-it':	0,
	'coriolanus':	1,
	'cymbeline':	0,
	'hamlet':	1,
	'henry-iv-part-1':	2,
	'henry-iv-part-2':	2,
	'henry-v':	2,
	'henry-viii':	2,
	'henry-vi-part-1':	2,
	'henry-vi-part-2':	2,
	'henry-vi-part-3':	2,
	'julius-caesar':	1,
	'king-john':	2,
	'king-lear':	1,
	'loves-labors-lost':	0,
	'macbeth':	1,
	'measure-for-measure':	0,
	'much-ado-about-nothing':	0,
	'othello':	1,
	'pericles':	0,
	'richard-ii':	2,
	'richard-iii':	2,
	'romeo-and-juliet':	1,
	'the-comedy-of-errors':	0,
	'the-merchant-of-venice':	0,
	'the-merry-wives-of-windsor':	0,
	'the-taming-of-the-shrew':	0,
	'the-tempest':	0,
	'the-two-gentlemen-of-verona':	0,
	'the-two-noble-kinsmen':	0,
	'the-winters-tale':	0,
	'timon-of-athens':	1,
	'titus-andronicus':	1,
	'troilus-and-cressida':	0,
	'twelfth-night':	0}
'''

labels=[]
k=0
hamlet_num=0
hamlet_popped_flag=1
for i in titles:
  labels.append(labels_orig[i])
  if i == 'hamlet':
    hamlet_num=k
  k+=1

In [ ]:
###
###テキスト分析3_セル5
###

print(labels)
print(titles)
print(hamlet_num)

In [ ]:
###
###テキスト分析3_セル6
###

#SentenceBERTによる分析
#SentenceBERTをロード
import numpy as np
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
#モデルにはMPNetを使う
mpnet = SentenceTransformer('all-mpnet-base-v2')
mpnet


In [ ]:
###
###テキスト分析3_セル7
###

#SentenceBERTを使って各劇作品の文章ベクトルを得る
embeddings = mpnet.encode(plays_short)

print(embeddings)
print(len(embeddings))

In [ ]:
###
###テキスト分析3_セル8
###

matrix=embeddings
matrix.shape

In [ ]:
###
###テキスト分析3_セル9
###

#hamletだけのぞく
print(hamlet_num)
#matrix_hamlet=matrix[hamlet_num].toarray()
#matrix2=matrix.toarray()
matrix_hamlet=matrix[hamlet_num]
matrix2=matrix
matrix3=np.delete(matrix2, hamlet_num,0)
if hamlet_popped_flag==1:
  popped_hamlet = labels.pop(hamlet_num)
  hamlet_popped_flag=0
print(len(labels))
print(len(matrix3))

In [ ]:
###
###テキスト分析3_セル10
###

#SVMを使う
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#データを分ける
input_train, input_test, output_train, output_test = train_test_split(matrix3, labels, test_size=0.1, random_state=None)


In [ ]:
###
###テキスト分析3_セル11
###

#SVMを学習
svm = SVC(kernel='linear', random_state=None)
svm.fit(input_train, output_train)

In [ ]:
###
###テキスト分析3_セル12
###

#できたモデルの精度の確認

pred_test = svm.predict(input_test)
accuracy_test = accuracy_score(output_test, pred_test)
print('テストデータの精度：',accuracy_test)


In [ ]:
###
###テキスト分析3_セル13
###

#hamletの予測
matrix_hamlet=matrix_hamlet.reshape(1, -1)
predict = svm.predict(matrix_hamlet)
predict_map=['喜劇','悲劇']
print('Hamletは'+predict_map[predict[0]]+'に分類しました。')